In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.transfermarkt.us/manchester-city/spielplandatum/verein/281'

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36"
}

In [4]:
response = requests.get(url, headers=headers)

In [5]:
response.status_code

200

In [6]:
soup = BeautifulSoup(response.content, "html.parser")

In [7]:
table = soup.select_one('div[class="responsive-table"]')

In [8]:
#Gathering the current headers in the table. There will be some added later.
headers = [th.text.strip() for th in table.find_all('th')]

In [9]:
rows = []
competition = []

#Retrieving the data from the html in the webpage.

for tr in table.find_all('tr'):
    row = [td.text.strip() for td in tr.find_all('td')]
    if len(row) == 1: #Using the html to find where the competition names are.
        comp = row
    elif len(row) > 9: #If there is more than just the competition name, there is data we need
        del row[5] #However, this spot contained a blank space where the logo for the team is on the webpage. Not needed.
        competition.append(comp)
    rows.append(row)

competition = [item[0].strip("[]") for item in competition]#Competition names were scraped with brackets around them. Not needed

In [10]:
#Creating the dataframe
data = []

for row in rows:
    if len(row)>1:
        data.append(row)
        
df = pd.DataFrame(data, columns=headers)
df.insert(0, "Competition", competition) #Inserting the competition list that we created earlier as a new column
df = df[(df['Result'] != '-:-')] #Removing the rows of the games not played yet.

In [11]:
#Adding a column for outcomes (Win/Draw/Loss)
outcome = []

results = df["Result"]

goals_home = results.str[:3].str.split(":").str[0].tolist()
goals_away = results.str[:3].str.split(":").str[1].tolist()

temp = 0

for n in df["Venue"]:
    if n == "H" and goals_home[temp] > goals_away[temp]:
        outcome.append('W')
        temp = temp + 1
    elif n == "H" and goals_home[temp] < goals_away[temp]: 
        outcome.append('L')
        temp = temp + 1
    elif n == "A" and goals_home[temp] > goals_away[temp]: 
        outcome.append('L')
        temp = temp + 1
    elif n == "A" and goals_home[temp] < goals_away[temp]: 
        outcome.append('W')
        temp = temp + 1
    else:
        outcome.append('D')
        temp = temp + 1

df.insert(10, "Outcome", outcome)
df

,Competition,Matchday,Date,Time,Venue,Ranking,Opponent,System of play,Attendance,Result,Outcome
0,Community Shield,Final,Sat 8/10/24,10:00 AM,A,,Man Utd,4-3-3 Attacking,78.146,7:8 on pens,W
1,Premier League,1,Sun 8/18/24,11:30 AM,A,(5.),Chelsea (4.),3-4-2-1,39.818,0:2,W
2,Premier League,2,Sat 8/24/24,10:00 AM,H,(4.),Ipswich (18.),3-4-2-1,55.017,4:1,W
3,Premier League,3,Sat 8/31/24,12:30 PM,A,(1.),West Ham (9.),3-4-2-1,62.500,1:3,W
4,Premier League,4,Sat 9/14/24,10:00 AM,H,(1.),Brentford (6.),4-2-3-1,55.017,2:1,W
5,UEFA Champions League,Group Stage,Wed 9/18/24,3:00 PM,H,,Inter,3-4-2-1,50.922,0:0,D
6,Premier League,5,Sun 9/22/24,11:30 AM,H,(1.),Arsenal (2.),4-3-3 Attacking,55.017,2:2,D
7,EFL Cup,Third Round,Tue 9/24/24,2:45 PM,H,,Watford,3-4-2-1,,2:1,W
8,Premier League,6,Sat 9/28/24,7:30 AM,A,(1.),Newcastle (6.),4-2-3-1,52.248,1:1,D
9,UEFA Champions League,Group Stage,Tue 10/1/24,3:00 PM,A,,Slo. Bratislava,4-2-3-1,22.500,0:4,W


In [12]:
#Find the winrate for home and away prem games
prem_df = df[df["Competition"] == "Premier League"]
home_games = prem_df[prem_df["Venue"] == "H"]
away_games = prem_df[prem_df["Venue"] == "A"]

# Count total matches
total_matches = len(prem_df)

# Count occurrences of each outcome
win_count = (prem_df['Outcome'] == 'W').sum()
draw_count = (prem_df['Outcome'] == 'D').sum()
loss_count = (prem_df['Outcome'] == 'L').sum()

# Count occurrences of each home outcome
home_win_count = (home_games['Outcome'] == 'W').sum()
home_draw_count = (home_games['Outcome'] == 'D').sum()
home_loss_count = (home_games['Outcome'] == 'L').sum()

# Count occurrences of each away outcome
away_win_count = (away_games['Outcome'] == 'W').sum()
away_draw_count = (away_games['Outcome'] == 'D').sum()
away_loss_count = (away_games['Outcome'] == 'L').sum()

# Calculate rates
win_rate = win_count / total_matches * 100
draw_rate = draw_count / total_matches * 100
loss_rate = loss_count / total_matches * 100

# Calculate home rates
win_rate_h = home_win_count / home_games.shape[0] * 100
draw_rate_h = home_draw_count / home_games.shape[0] * 100
loss_rate_h = home_loss_count / home_games.shape[0] * 100

# Calculate away rates
win_rate_a = away_win_count / away_games.shape[0] * 100
draw_rate_a = away_draw_count / away_games.shape[0] * 100
loss_rate_a = away_loss_count / away_games.shape[0] * 100

# Print results
print(f"Overall Win Rate: {win_rate:.2f}%")
print(f"Overall Draw Rate: {draw_rate:.2f}%")
print(f"Overall Loss Rate: {loss_rate:.2f}%")
print("--------------")
print(f"Home Win Rate: {win_rate_h:.2f}%")
print(f"Home Draw Rate: {draw_rate_h:.2f}%")
print(f"Home Loss Rate: {loss_rate_h:.2f}%")
print("--------------")
print(f"Away Win Rate: {win_rate_a:.2f}%")
print(f"Away Draw Rate: {draw_rate_a:.2f}%")
print(f"Away Loss Rate: {loss_rate_a:.2f}%")

Overall Win Rate: 50.00%
Overall Draw Rate: 20.83%
Overall Loss Rate: 29.17%
--------------
Home Win Rate: 63.64%
Home Draw Rate: 18.18%
Home Loss Rate: 18.18%
--------------
Away Win Rate: 38.46%
Away Draw Rate: 23.08%
Away Loss Rate: 38.46%


In [13]:
#I want to find if Man City has a higher winrate for early games(before 12pm) or late games(after 12pm).
early_games = df[df['Time'].str[-2:] == 'AM']
late_games = df[df['Time'].str[-2:] == 'PM']

early_win_count = (early_games['Outcome'] == 'W').sum()
late_win_count = (late_games['Outcome'] == 'W').sum()

early_win_rate = early_win_count / len(early_games) * 100
late_win_rate = late_win_count / len(late_games) * 100

print(f"Early Win Rate: {early_win_rate:.2f}%")
print(f"Late Win Rate: {late_win_rate:.2f}%")

Early Win Rate: 55.00%
Late Win Rate: 44.44%
